## Setup the environment (database, schema, stage, warehouse, etc)
+ Copy the below setup into Snowflake Worksheet to initiated the environment 

In [ ]:
-- to enable cortex on the unsported region
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US'; 

-- Create FWDDEMO database
CREATE OR REPLACE DATABASE FWDDEMO;

-- Create schema
CREATE OR REPLACE SCHEMA DEMO;

CREATE STAGE STG_DEMO;

CREATE OR REPLACE WAREHOUSE CORTEX_ANALYST_WH;
WAREHOUSE_SIZE = 'LARGE'  
AUTO_SUSPEND = 60;



## Create view tables for SP_500 and SP_500_VISITS

- To create the SP_500 view table:
    - Upload the SP_500 file (referential data) into FWDDEMO.PUBLIC
    - Upload the S&P_500_Finance_Info file (referential data) into FWDDEMO.PUBLIC
- To create the SP_500_VISITS view table:
    - Go to Snowflake Marketplace and search this data **"S & P 500 by domain and aggregated by tickers (Sample)"**
    - Click get data and create a new databset `SP_500_BY_DOMAIN` to store the new dataset from marketplace


In [ ]:
-- Create SP_500 view table
create or replace view FWDDEMO.PUBLIC."SP_500_UPDATE" as
select distinct 
    a.SYMBOL as TICKER,  
    COALESCE(a.SECURITY, b.NAME) AS NAME,
    b.SECTOR, 
    a.GICS_SUB_INDUSTRY as INDUSTRY, 
    a.HEADQUARTERS_LOCATION as HEADQUARTERS, 
    a.DATE_ADDED, 
    a.CIK, 
    a.FOUNDED,
    b.PRICE, 
    b.PRICE_EARNINGS, 
    b.DIVIDEND_YIELD, 
    b.EARNING_SHARE as EARNINGS_SHARE, 
    b."52_Week_Low" as "LOW_52_WEEKS", 
    b."52_Week_High" as "HIGH_52_WEEKS", 
    b.MARKET_CAP, 
    b.EBITDA, 
    b."Price_Sales" as "PRICE_SALES", 
    b."Price_Book" as "PRICE_BOOK", 
    b.SEC_FILINGS
from FWDDEMO.PUBLIC."SP_500" a
left join FWDDEMO.PUBLIC."S&P_500_Finance_Info" b ON a.SYMBOL = b.SYMBOL;

--Create SP_500_VISITS view table

create or replace view FWDDEMO.PUBLIC."SP_500_VISITS" as
select
TICKER,
COMPANY_NAME,  
DOMAIN, 
DATE::DATE as DATE, 
COUNTRY, 
DESKTOP_VISITS::FLOAT as DESKTOP_VISITS, 
DESKTOP_BOUNCE_RATE::FLOAT as DESKTOP_BOUNCE_RATE, 
DESKTOP_AVG_VISIT_DURATION::FLOAT as DESKTOP_AVG_VISIT_DURATION, 
DESKTOP_PAGES_PER_VISIT::FLOAT as DESKTOP_PAGES_PER_VISIT,
MOBILE_VISITS::FLOAT as MOBILE_VISITS, 
MOBILE_BOUNCE_RATE::FLOAT as MOBILE_BOUNCE_RATE, 
MOBILE_AVG_VISIT_DURATION::FLOAT as MOBILE_AVG_VISIT_DURATION, 
MOBILE_PAGES_PER_VISIT::FLOAT as MOBILE_PAGES_PER_VISIT, 
TOTAL_VISITS::FLOAT as TOTAL_VISITS, 
TOTAL_BOUNCE_RATE::FLOAT as TOTAL_BOUNCE_RATE, 
TOTAL_AVG_VISIT_DURATION::FLOAT as TOTAL_AVG_VISIT_DURATION, 
TOTAL_PAGES_PER_VISIT::FLOAT as TOTAL_PAGES_PER_VISIT
from SP_500_BY_DOMAIN.DATAFEEDS.SP_500;

## S&P 500 Fuzzy Matching

### 📦 Package Overview

This package enables interactive data apps using **Streamlit**, with data processing via **Pandas** and **Snowflake Snowpark**, and fuzzy matching powered by **RapidFuzz**.

#### 🔧 Imports:
- `pandas` – for data manipulation
- `streamlit` – for building the app interface
- `snowflake.snowpark.context` – to connect and run queries in Snowflake
- `rapidfuzz.fuzz` – for efficient string similarity matching

#### Fuzzy Matching 
+ Using the security column (company name) to perform fuzzy matching with the article content
+ For this DEMO, we define the threshold = 85

In [ ]:
import pandas as pd
import streamlit as st
from snowflake.snowpark.context import get_active_session
from rapidfuzz import fuzz

# Get Snowflake session
session = st.connection('snowflake').session()

# Fetch article data
def fetch_data_from_snowflake():
    snowflake_table = session.table("FWDDEMO.PUBLIC.ARTICLE_CONTENT_SCRAPED")
    return snowflake_table.to_pandas()

# Fetch S&P 500 SECURITY and SYMBOL
def fetch_sp500_data():
    snowflake_table = session.table("FWDDEMO.PUBLIC.SP_500")
    return snowflake_table.to_pandas()[["SECURITY", "SYMBOL"]].dropna()

# Fuzzy match based on SECURITY, return all matching SYMBOLs
def fuzzy_match_symbols_only(content, sp500_df, threshold=85):
    content_lower = content.lower()
    symbols = []

    for _, row in sp500_df.iterrows():
        name = row["SECURITY"]
        symbol = row["SYMBOL"]
        score = fuzz.partial_ratio(name.lower(), content_lower)

        if score >= threshold:
            symbols.append(symbol)

    return list(set(symbols)) if symbols else None

# --- Process Data ---
data = fetch_data_from_snowflake()
sp500_df = fetch_sp500_data()

# Apply fuzzy matching to content and return only matching symbols
data["SP_500_Symbol"] = data["content_scraped"].apply(
    lambda x: fuzzy_match_symbols_only(x, sp500_df) if pd.notnull(x) else None
)

# --- Display in Streamlit ---
st.dataframe(data)

# --- Save back to Snowflake ---
snowpark_df = session.create_dataframe(data)
snowpark_df.write.mode("overwrite").save_as_table("FWDDEMO.PUBLIC.ArticleContentScraped_Matched")

st.success("✅ Fuzzy-matched SYMBOLs saved to ArticleContentScraped_Matched!")


### Building a sentiment analysis for the article content using VADER_LEXICON dictionary 

#### 🔧 Imports:
- `regex` – work with Regular Expressions

#### Sentiment Labeling:

+ Scores ≤ -0.05 are labeled as "Negative"
+ Scores ≥ 0.05 are labeled as "Positive"
+ Scores between -0.05 and 0.05 are labeled as "Neutral"

In [ ]:
import streamlit as st
from snowflake.snowpark import Session
import pandas as pd
import re

# Step 1: Get Snowflake session
session = st.connection('snowflake').session()

# Check table structure first
table_columns = session.sql("SELECT * FROM FWDDEMO.PUBLIC.ARTICLECONTENTSCRAPED_MATCHED LIMIT 1").schema.names
#st.write("Available columns:", table_columns)

# Step 2: Load vader_lexicon from Snowflake table
lexicon_df = session.sql("SELECT $1, $2 FROM FWDDEMO.PUBLIC.VADER_LEXICON").to_pandas()

# Optional: Display loaded lexicon
#st.write("Lexicon Preview", lexicon_df.head())

# Step 3: Build lexicon dictionary
lexicon_dict = dict(zip(lexicon_df['$1'], lexicon_df['$2']))

# Step 4: Define a simple sentiment analyzer using the lexicon
class CustomVaderSentimentAnalyzer:
    def __init__(self, lexicon_dict):
        self.lexicon = lexicon_dict
        
    def polarity_scores(self, text):
        """
        Return a float for sentiment strength based on the lexicon.
        Positive values are positive sentiment, negative value are negative sentiment.
        """
        # Handle None or empty text
        if text is None or not isinstance(text, str):
            return {'compound': 0, 'pos': 0, 'neg': 0, 'neu': 1}
            
        text = text.lower()
        words = re.findall(r'\b\w+\b', text)  # Simple tokenization
        
        total_score = 0
        word_count = 0
        
        for word in words:
            if word in self.lexicon:
                total_score += self.lexicon[word]
                word_count += 1
        
        # Calculate compound score (normalized between -1 and 1)
        compound = 0
        if word_count > 0:
            compound = total_score / (word_count + 0.1)  # Add small value to avoid division by zero
            # Normalize between -1 and 1
            compound = max(min(compound, 1.0), -1.0)
            
        # Return dict format similar to VADER
        return {
            'compound': compound,
            'pos': max(0, compound),
            'neg': max(0, -compound),
            'neu': 1 - abs(compound)
        }

# Instantiate analyzer with your lexicon
analyzer = CustomVaderSentimentAnalyzer(lexicon_dict)

# Step 5: Define sentiment labeling logic
def get_sentiment_label(score):
    if score <= -0.05:
        return 'Negative'
    elif score >= 0.05:
        return 'Positive'
    else:
        return 'Neutral'

# Step 6: Fetch articles with updated column handling
def fetch_data_from_snowflake():
    # Get all columns from the table
    snowflake_table = session.table("FWDDEMO.PUBLIC.ARTICLECONTENTSCRAPED_MATCHED")
    df = snowflake_table.to_pandas()
    
    # Display the column names to help debug
    #st.write("Columns in dataframe:", df.columns.tolist())
    return df

article_df = fetch_data_from_snowflake()

# Use first string column for sentiment analysis if content_scraped doesn't exist
def find_text_column(df):
    for col in df.columns:
        # Check if column exists and contains string data
        if df[col].dtype == 'object':
            sample = df[col].iloc[0] if not df.empty else None
            if isinstance(sample, str):
                return col
    return None

text_column = 'content_scraped' if 'content_scraped' in article_df.columns else find_text_column(article_df)

if text_column:
    st.write(f"Using column '{text_column}' for sentiment analysis")
    
    # Step 7: Apply sentiment scoring with error handling
    def safe_sentiment_analysis(text):
        try:
            if pd.isna(text) or text is None or not isinstance(text, str):
                return 0  # Neutral sentiment for invalid texts
            return analyzer.polarity_scores(text)['compound']
        except Exception as e:
            st.warning(f"Error processing text: {e}")
            return 0

    # Apply sentiment scoring with error handling
    article_df['article_sentiment'] = article_df[text_column].apply(safe_sentiment_analysis)
    article_df['sentiment_label'] = article_df['article_sentiment'].apply(get_sentiment_label)

    # --- Save back to Snowflake ---
    snowpark_df = session.create_dataframe(article_df)
    snowpark_df.write.mode("overwrite").save_as_table("FWDDEMO.PUBLIC.ArticleContentScraped_Sentiment")
    
    # Step 8: Display result
    #st.dataframe(article_df[[text_column, 'article_sentiment', 'sentiment_label']])
    st.dataframe(article_df)
else:
    st.error("No suitable text column found for sentiment analysis")

### Alter column SP_500_Symbol from varchar to varant 
- Copy the code below into Snoflake worksheet

In [ ]:
-- Alter column SP_500_Symbol from varchar to varant 
ALTER TABLE FWDDEMO.PUBLIC.ARTICLECONTENTSCRAPED_SENTIMENT
ADD COLUMN SP_500_Symbol_variant VARIANT;

UPDATE FWDDEMO.PUBLIC.ARTICLECONTENTSCRAPED_SENTIMENT
SET SP_500_Symbol_variant = TO_VARIANT("SP_500_Symbol");

UPDATE FWDDEMO.PUBLIC.ARTICLECONTENTSCRAPED_SENTIMENT
SET SP_500_Symbol_variant = PARSE_JSON("SP_500_Symbol");

ALTER TABLE FWDDEMO.PUBLIC.ARTICLECONTENTSCRAPED_SENTIMENT
DROP COLUMN "SP_500_Symbol";

ALTER TABLE FWDDEMO.PUBLIC.ARTICLECONTENTSCRAPED_SENTIMENT
RENAME COLUMN SP_500_Symbol_variant TO SP_500_Symbol;

### Create a vew table for the SP_500_articles 

- This table consist of the new columns from fuzzy matching and sentiment analysis

In [ ]:
--create or replace view FWDDEMO.PUBLIC."SP_500_ARTICLES" as
select 
    "source" as SOURCE, 
    "title" as ARTICLE_TITLE, 
    "description" as ARTICLE_SUMMARY, 
    "url" as ARTICLE_URL, 
    "publishedAt"::DATE as PUBLISH_DATE,
    "content_scraped" as ARTICLE_CONTENT, 
    value::text AS TICKER,
    "article_sentiment"::float as SENTIMENT_SCORE,
    "sentiment_label" as SENTIMENT
from
  FWDDEMO.PUBLIC.ARTICLECONTENTSCRAPED_SENTIMENT src,
  LATERAL FLATTEN (INPUT => "SP_500_SYMBOL") AS _flattened (SEQ, KEY, PATH, INDEX, VALUE, THIS);